Attempts to incorporate esp32 Ultra Low Power processor 
compilation into the jupyter micropython notebook 

https://github.com/micropython/micropython/blob/master/ports/esp32/README.ulp.md


We also need the whole esp32 micropython toolchain
https://github.com/micropython/micropython/tree/master/ports/esp32

and git clone https://github.com/espressif/esp-idf.git
git checkout 30545f4cccec7460634b656d278782dd7151098e
git submodule update --init --recursive

Added in the $ESPIDF environment variable.  
Did not do the binary part of the toolchain.  

Now looking at:
https://github.com/ThomasWaldmann/py-esp32-ulp/


In [150]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
ets Jun  8 2016 00:22:57

rst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:2
load:0x3fff0018,len:4
load:0x3fff001c,len:4732
load:0x40078000,len:7496
load:0x40080400,len:5512
entry 0x4008114c
OSError: [Errno 2] ENOENT
MicroPython v1.9.4-771-gb33f108cd on 2018-12-30; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


In [12]:
%sendtofile --mkdir --source /home/julian/extrepositories/py-esp32-ulp/esp32_ulp  esp32_ulp

Sent 283 lines (9521 bytes) to esp32_ulp/assemble.py.


In [548]:
from esp32_ulp.__main__ import src_to_binary
from esp32 import ULP

source = """\
data:       .long 0x39000102
data2:      .long 0x11223344
entry:      rsh r3, r3, 16
            move r3, data    # load address of data into r3
            ld r2, r3, 2     # load data contents ([r3+0]) into r2
rep:        add r2, r2, 8    # increment r2
            st r2, r3, 2     # store r2 contents into data ([r3+0])
            #halt             # halt ULP co-prozessor (until it gets waked up again)
            #jump rep
kkk:        halt
"""
ulp = ULP()
binary = src_to_binary(source)
ulp.load_binary(0, binary)


0000 data
0001 data2
0002 entry
0005 rep
0007 kkk


In [543]:
ulp.set_wakeup_period(0, -1)

In [555]:
ulp.run(2*4)
ulp.set_wakeup_period(0, 5000000)

In [580]:
from machine import mem32
print(hex(mem32[0x50000000]))
print(hex(mem32[0x50000004]))

0xc3011a
0x11223344


In [126]:
from machine import mem8
print(hex(mem8[0x50000015]))


0x0


In [45]:
help(ulp)

object <ULP> is of type ULP
  set_wakeup_period -- <function>
  load_binary -- <function>
  run -- <function>
  RESERVE_MEM -- 2040


In [587]:
print(p)

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'p' isn't defined


In [585]:
mem32[0x3ff44000 + 0xC] ^= 1 << 2

In [586]:
mem32[0x3ff44000 + 0x8] ^= 1 << 2

In [29]:
from esp32 import ULP
from machine import mem32

load_addr, entry_addr = 0, 4

ULP_MEM_BASE = 0x50000000
ULP_DATA_MASK = 0xffff  # ULP data is only in lower 16 bits

ulp = ULP()
#ulp.set_wakeup_period(0, 50000)  # use timer0, wakeup after 50.000 cycles
ulp.load_binary(load_addr, binary)

mem32[ULP_MEM_BASE + load_addr] = 0x1000
ulp.run(entry_addr)



In [ ]:
hex(mem32[ULP_MEM_BASE + load_addr] & ULP_DATA_MASK)

In [605]:
import time
#for i in range(50):
#    print(time.ticks_ms(), hex(mem32[ULP_MEM_BASE + load_addr] & ULP_DATA_MASK))


In [668]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [670]:
from machine import Pin, mem32, mem8
import time
ulp = ULP()


In [677]:
p = Pin(2, Pin.OUT)

In [591]:
p.value(0)

In [594]:
p.value(0)
mem32

In [606]:
print(hex(mem32[0x3ff44000 + 0x8]))
mem32[0x3ff44008] = 1 << 2
print(hex(mem32[0x3ff44000 + 0x8]))

0x0
0x0


In [609]:
#mem32[0x3ff44008] = 1 << 2
mem32[0x3ff4400C] = 1 << 2


In [ ]:
from machine import mem32

In [687]:
ulp.set_wakeup_period(0, -1)

In [684]:
from esp32_ulp.__main__ import src_to_binary
from esp32 import ULP

source = """\
data:       .long 0x39000102
data2:      .long 0x11223344
entry:      rsh r3, r3, 16

            reg_wr  0x3ff4840C, 26, 26, 1
            reg_wr  0x3ff48410, 26, 26, 1
            reg_wr  0x3ff48414, 26, 26, 1

            reg_wr  0x3ff48404, 26, 26, 1
                        
            move r3, data    # load address of data into r3
            ld r2, r3, 2     # load data contents ([r3+0]) into r2
rep:        add r2, r2, 8    # increment r2
            st r2, r3, 2     # store r2 contents into data ([r3+0])
            #jump rep
kkk:        halt
"""
ulp = ULP()
binary = src_to_binary(source)
ulp.load_binary(0, binary)


.0000 data
0001 data2
0002 entry
0009 rep
0011 kkk


In [686]:
ulp.run(2*4)
ulp.set_wakeup_period(0, 5000000)

In [688]:
from machine import mem32
print(hex(mem32[0x50000000]))
print(hex(mem32[0x50000004]))

0x1430152
0x11223344


In [ ]:
#REG_WR      0x120, 2, 2, 1   
#// set 8 bits: REG[0x120][7:0] = 0x10
#mem32[0x3ff44008] = 1 << 2
#mem32[0x3ff4400C] = 1 << 2


In [678]:
mem32[0x3ff44008] = 4
print(hex(mem32[0x3ff44004]))
time.sleep(1)
mem32[0x3ff4400C] = 4
print(hex(mem32[0x3ff44004]))


0x4
0x0


In [695]:
mem32[0x3ff4840C] = 1<<(12+14)
print(hex(mem32[0x3ff4840C]))

0x4000000


In [696]:
mem32[0x3ff4840C] = 1<<(12+14)
mem32[0x3ff48410] = 1<<(12+14)
mem32[0x3ff48414] = 1<<(12+14)

In [697]:
print(hex(mem32[0x3ff48410]))

0x0


In [694]:
mem32[0x3ff48404] = 1<<(12+14)
print(hex(mem32[0x3ff48400]))
time.sleep(1)
mem32[0x3ff48408] = 1<<(12+14)
print(hex(mem32[0x3ff48400]))

0xc000000
0x8000000


In [698]:
print(hex(mem32[0x3FF48458]))


0x0


In [700]:
import machine
help(machine)

object <module 'umachine'> is of type module
  __name__ -- umachine
  mem8 -- <8-bit memory>
  mem16 -- <16-bit memory>
  mem32 -- <32-bit memory>
  freq -- <function>
  reset -- <function>
  unique_id -- <function>
  sleep -- <function>
  deepsleep -- <function>
  idle -- <function>
  disable_irq -- <function>
  enable_irq -- <function>
  time_pulse_us -- <function>
  Timer -- <class 'Timer'>
  WDT -- <class 'WDT'>
  SLEEP -- 2
  DEEPSLEEP -- 4
  Pin -- <class 'Pin'>
  Signal -- <class 'Signal'>
  TouchPad -- <class 'TouchPad'>
  ADC -- <class 'ADC'>
  DAC -- <class 'DAC'>
  I2C -- <class 'I2C'>
  PWM -- <class 'PWM'>
  RTC -- <class 'RTC'>
  SPI -- <class 'SoftSPI'>
  UART -- <class 'UART'>
  reset_cause -- <function>
  HARD_RESET -- 2
  PWRON_RESET -- 1
  WDT_RESET -- 3
  DEEPSLEEP_RESET -- 4
  SOFT_RESET -- 5
  wake_reason -- <function>
  PIN_WAKE -- 1
  EXT0_WAKE -- 1
  EXT1_WAKE -- 2
  TIMER_WAKE -- 3
  TOUCHPAD_WAKE -- 4
  ULP_WAKE -- 5


In [701]:
import esp
help(esp)

object <module 'esp'> is of type module
  __name__ -- esp
  osdebug -- <function>
  flash_read -- <function>
  flash_write -- <function>
  flash_erase -- <function>
  flash_size -- <function>
  flash_user_start -- <function>
  gpio_matrix_in -- <function>
  gpio_matrix_out -- <function>
  neopixel_write -- <function>
  dht_readinto -- <function>
  LOG_NONE -- 0
  LOG_ERROR -- 1
  LOG_WARNING -- 2
  LOG_INFO -- 3
  LOG_DEBUG -- 4
  LOG_VERBOSE -- 5


In [704]:
x = Pin(2)
print(x)

Pin(2)
